# Part 2 Question 1 - Cracking the password

## Open Zipfile

In [5]:
from zipfile import ZipFile

In [2]:
filename = "Group A_CYBE_HW1-P2_64bit.zip"

In [74]:
# A fail example
ZipFile(filename).extractall(pwd="random")

TypeError: pwd: expected bytes, got str

## Construct Password

- Password have 8 characters
- Password contain one or more words (1-2) from the below word list
- Password might contain numeric digits and non-alphanumeric \@ and \$

In [19]:
words = ["CWRU", "CDS", "Case", "Janie", "Jack",
         "Puppy", "Love", "Euclid", "Ohio", "Paris"]

## Construct Word Tuple

A word tuple is a selection of words that might fit into one password from the above list.

In [73]:
from itertools import combinations, combinations_with_replacement, permutations
from functools import reduce


def strtuplen(tup):
    # takes a tuple of string and return total length
    if len(tup) == 1:
        return len(tup[0])
    return reduce(lambda x, y: len(x)+len(y), tup)

# If the word tuple has length exceed 8, discard
MAX_PWLEN = 8
def strtuplenok(tup): return tup if strtuplen(tup) < MAX_PWLEN else None
word_tups = list(map(lambda x: (x,), words))
word_tups.extend(filter(strtuplenok, combinations(words, 2)))

## Construct Remaining Characters

In [67]:
char_list = [c for c in "0123456789@$"]

In [68]:
def strtupcombo(tup):
    # for a given word tuples, the rest are digits and special characters @ and $
    # returns all the repeat combaintions
    n_chars = MAX_PWLEN - strtuplen(tup)
    return combinations_with_replacement(char_list, n_chars)

## Crack!


In [70]:
from joblib import parallel_backend, Parallel, delayed

def is_zip_password(zf, pw):
    # check if it is a password for the zipfile
    try:
        zf.extractall(pwd=pw)
    except:
        return False
    return True

def run_crack(zf, all_tups):
    # run the crack on the specific zipfile and word_tuple list
    def __check_individual_tup(tup):
        for combo in strtupcombo(tup):
            for tupcombo in permutations((*tup, *combo)):
                pwstr = "".join(tupcombo)
                if is_zip_password(zf, pwstr.encode('cp437')):
                    return pwstr
        return None
    
    with parallel_backend("threading", n_jobs=min(len(all_tups), 3)):
        results = Parallel()(delayed(__check_individual_tup)(t) for t in all_tups)
    return results

In [72]:
# This takes an hour to run!!!!!
run_crack(ZipFile(filename), word_tups)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 '5@Paris$',
 None,
 None,
 None,
 None,
 None]